# Esercizio 1. Predizione della qualità di un vino partendo da risultati di test fisico-chimici mediante percettrone multi-strato
Con riferimento all'esercizio tutorial descritto in 'neural_networks_regression.ipynb', risolvere il problema affrontato con regressione mediante classificazione. 

Per fare ciò occorre modificare:

*   encoding delle etichette (one-hot encoding, vedi sotto), sia in fase di training sia di testing 
*   lo strato di uscita con un neurone per classe (11 in tot), attivati mediante softmax (activation='softmax')
*   funzione costo (tf.keras.losses.CategoricalCrossentropy())
*   metriche di performance (tf.keras.metrics.CategoricalAccuracy())
*   (class weighting per dataset sbilanciati)





In [ ]:
X = [1, 3, 7, 1, 0, ..., 9] # 11 features
yvera = 4 # dense label coding
yp = [0.1, 0., 0.2, 0.6, ..., ] # probabilità: sommano ad 1, supponiamo alla prima iterazione
yp_ = [0.0001, 0.001, 0.00002, 0.89, ..., ]# all'ultima iterazione

# ciascun valore mi dice la probabilità che l'ingresso X appartenga ad una certa classe 
yvera_prb = [0, 0, 0, 1, 0, 0, ... 0] # (one-hot encoding) vettori a 11 dimen.
# minimizzare la cross-entropy tra yvera_prb, yp equivale a minimizzare la
# distanza tra le distribuzioni di probabilità. Voi le potete vedere come distanze vettoriali
# tra il vettore ad 11 dim yp e vettore yvera_prb

# distanza tra distr. proba: distranza di Kullback-leibler 

y_train # (1300,) [0, 3, 7, 1, 5, 9,...]
y_train_onehot # (1300, 11), dove ciascuna riga ha la versione one-hot dell'etichetta associata
y_test
y_test_onehot

# acc: no. esempi correttamente classificati / no. totale di esempi
# confusion matrix (matrice di confusione), da questa vengono poi definite tante metriche di prestazione (acc, F1 score,...)
# esistono altre metriche speicifhe per dataset sbilanciati come F1 score

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import matplotlib.pyplot as plt
import math

# one-hot encoding
# supponendo di aver caricato y_train e y_test...
y_train_onehot = np.zeros((y_train.shape[0], 11))
y_train_onehot[np.arange(y_train.shape[0]), y_train] = 1

y_test_onehot = np.zeros((y_test.shape[0], 11))
y_test_onehot[np.arange(y_test.shape[0]), y_test] = 1

# Esercizio 2. Classificazione di caratteri scritti a mano su carta (handwritten digit classification) mediante rete neurale convoluzionale

Il compito di classificazione obiettivo consiste nell'associare l'etichetta corretta a numeri scritti a mano riportati in un'immagine gray-scale di dimensioni 28x28. Quindi, essendoci 10 possibili digit (0,...,9) il compito consiste nella discriminazione di 10 possibili classi.

Il dataset è disponibile pubblicamente online (http://yann.lecun.com/exdb/mnist/) ma utilizzeremo le funzioni di keras per caricarlo in memoria (vedi sotto). 

Utilizzare una rete neurale convoluzionale (CNN) per risolvere il problema di classificazione. Definire la seguente sequenza di strati:


INPUT-CONV-MAXPOOL-DROPOUT-CONV-MAXPOOL-DROPOUT-FLATTEN-DENSE(OUTPUT)


Definire gli strati intermedi con: 
 
*   Conv2D(16, kernel_size=(5, 5), in cui viene eseguita la convoluzione con 16 filtri convoluzionali addestrabili 5x5
*   MaxPooling2D(pool_size=(2, 2), in cui viene eseguito l'operatore di max pooling per dimezzare le dimensioni spaziali (ridurre computazioni)
*   Dropout(0.25), operatore di dropout (dropout rate=0.25)
*   Flatten()

Vedete anche i seguenti link per comprendere meglio il funzionamento di CNN per classificazione di digits:
https://adamharley.com/nn_vis/cnn/2d.html.

Addestrare e testare il modello per risolvere il problema di classificazione, analogamente a quanto fatto per l'esercizio 1. 

N.B. visto che ci sono molti esempi, valutare di addestrare per poche epoche (es. fino 50) con mini-batch size elevati (es. 128 o 256).

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import matplotlib.pyplot as plt
import math

# Prepare the dataset.
# load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # uint8 (0-255)
N = 28 # number of rows and columns of the input square matrix
C = 10 # number of total classes (e.g., 10 possible digits)

print("Input data type:", x_train.dtype)
print("Min value:", x_train.min())
print("Max value:", x_train.max())
print("Shape of the training examples:", x_train.shape)
print("Shape of the test examples:", x_test.shape)

plt.hist(y_train)
plt.show()

y_train_onehot = np.zeros((y_train.shape[0], 10))
y_train_onehot[np.arange(y_train.shape[0]), y_train] = 1

y_test_onehot = np.zeros((y_test.shape[0], 10))
y_test_onehot[np.arange(y_test.shape[0]), y_test] = 1
print(y_test[0])
print(y_test_onehot[0,:])

mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)
x_train = x_train-mean_train
x_train = x_train/(std_train+1e-16)
x_test = x_test-mean_train
x_test = x_test/(std_train+1e-16)


input_shape = (N, N, 1) # input dimension (grey-scale image 28 x 28, represented as a single input feature map 28x28x1)
inputs = keras.Input(shape=input_shape, name="input_features")
x1 = layers.Conv2D(16, kernel_size=(5, 5),  activation="relu",
                   kernel_regularizer=regularizers.L2(1e-4),
                   bias_regularizer=regularizers.L2(1e-4),
                   name='conv_layer0')(inputs)
x2 = layers.MaxPooling2D(pool_size=(2, 2),name='maxpool0')(x1)
x3 = layers.Dropout(0.25,name='dropout0')(x2)

x4 = layers.Conv2D(16, kernel_size=(5, 5),activation="relu",
                   kernel_regularizer=regularizers.L2(1e-4),
                   bias_regularizer=regularizers.L2(1e-4),
                   name='conv_layer1')(x3)
x5 = layers.MaxPooling2D(pool_size=(2, 2),name='maxpool1')(x4)
x6 = layers.Dropout(0.25,name='dropout1')(x5)

x7 = layers.Flatten()(x6)
outputs = layers.Dense(C, activation='softmax', name='dense0')(x7)
model = keras.Model(inputs=inputs, outputs=outputs, name='CNN')
model.summary()